# Music Key Estimation
Based off the work of Schreiber and Müller. Full citation follows.
> Hendrik Schreiber and Meinard Müller, [Musical Tempo and Key Estimation using Convolutional Neural Networks with Directional Filters](http://smc2019.uma.es/articles/P1/P1_07_SMC2019_paper.pdf), In Proceedings of the Sound and Music Computing Conference (SMC), Málaga, Spain, May 2019.


## Imports

In [1]:
import os
import pathlib

import pandas

## Constants


In [2]:
DATA_FOLDER = pathlib.Path("data")
AUDIO_DATA = pathlib.Path("data/audio")

## Obtaining Data


Let's get all the audio files within the data folder.

In [3]:
allFiles = os.listdir(AUDIO_DATA)

Let's get only the audio files present in the folder.


In [4]:
audioFiles = [x for x in allFiles if x.lower().endswith(".mp3")]

Let's look at some of the audio files present.

In [5]:
audioFiles

['74680.LOFI.mp3',
 '608572.LOFI.mp3',
 '5061.LOFI.mp3',
 '180779.LOFI.mp3',
 '299807.LOFI.mp3',
 '297886.LOFI.mp3',
 '1492875.LOFI.mp3',
 '4857871.LOFI.mp3',
 '6529435.LOFI.mp3',
 '3656502.LOFI.mp3']

Now let's get the key annotations of the audio files.

The key annotations are stored in a file named `annotations.txt`. However, it is formatted like a tab-separated values (TSV) file. Thus we shall read it in as a TSV file and process it.


In [6]:
allAnnotations = pandas.read_csv(DATA_FOLDER / "annotations.txt", sep="\t")

Let's analyse the data.


In [7]:
allAnnotations

,ID,MANUAL KEY,C
0,5061,D# minor,2
1,5881,A major,2
2,7217,F minor,2
3,10359,D minor,2
4,10981,-,0
...,...,...,...
1481,6532021,C# minor,1
1482,6535130,E minor,2
1483,6537998,E major,2
1484,6563157,B minor,2


Some things to note about the data:
- Some rows do not have a `MANUAL KEY`. These rows need to be removed.
- The `C` value is the *confidence value* of the key estimate. Higher confidence values means that the key value is more likely to be correct.
    - Interestingly, rows without a manual key have a confidence value of `0`. This reveals a quick way to remove non-key rows.
- Some rows have more than one `MANUAL KEY` (see row 47 with `ID = 61578`). For consistency, we shall pick the first key that appears in that `MANUAL KEY` column.

After cleaning up the data, we can remove the `C` column as it is not helpful when we are estimating the key of the music.

Let's run the processes described above.


In [8]:
# Handle the rows with more than one manual key
numRows = len(allAnnotations)

for index in range(numRows):
    # Get the "MANUAL KEY" column's value
    manualKey = allAnnotations.iloc[index]["MANUAL KEY"]

    # Split the key entry by a forward slash (/)
    manualKeys = manualKey.split("/")

    # Keep only the first key
    manualKey = manualKeys[0]

    # Update the dataframe, by adding a new "KEY" column in place of the "MANUAL KEY" column
    allAnnotations.loc[index, "KEY"] = manualKey

# Remove all rows with a confidence value of 0 (i.e. no manual key)
allAnnotations.drop(allAnnotations[allAnnotations["C"] == 0].index, inplace=True)

# Remove the "C" and "MANUAL KEY" column
allAnnotations.drop(columns=["C", "MANUAL KEY"], inplace=True)

Let's look at the updated data.


In [9]:
allAnnotations

,ID,KEY
0,5061,D# minor
1,5881,A major
2,7217,F minor
3,10359,D minor
5,11077,G minor
...,...,...
1481,6532021,C# minor
1482,6535130,E minor
1483,6537998,E major
1484,6563157,B minor


Now we have to reindex the rows of the dataframe, as some rows have been removed during the data augmentation step above.


In [10]:
allAnnotations.reset_index(drop=True, inplace=True)  # We don't want the old index being added to the dataframe
allAnnotations

,ID,KEY
0,5061,D# minor
1,5881,A major
2,7217,F minor
3,10359,D minor
4,11077,G minor
...,...,...
1385,6532021,C# minor
1386,6535130,E minor
1387,6537998,E major
1388,6563157,B minor


We now need to keep only the IDs where we have both an audio file and an annotation match.


In [11]:
# Get all IDs
audioFileIDs = set(int(x.split(".")[0]) for x in audioFiles)
annotationFileIDs = set(allAnnotations["ID"].to_list())

# Find their intersection
relevantIDs = audioFileIDs.intersection(annotationFileIDs)

What are these relevant IDs?


In [12]:
relevantIDs

{5061, 74680, 299807, 608572, 1492875, 4857871, 6529435}

Now keep audio files and annotation data that is relevant to us.

In [13]:
relevantAudioFiles = []
for filename in audioFiles:
    if int(filename.split(".")[0]) in relevantIDs:
        relevantAudioFiles.append(filename)

relevantAnnotations = allAnnotations[allAnnotations["ID"].isin(relevantIDs)]
relevantAnnotations.reset_index(drop=True, inplace=True)

Display this data.

In [14]:
relevantAudioFiles

['74680.LOFI.mp3',
 '608572.LOFI.mp3',
 '5061.LOFI.mp3',
 '299807.LOFI.mp3',
 '1492875.LOFI.mp3',
 '4857871.LOFI.mp3',
 '6529435.LOFI.mp3']

In [15]:
relevantAnnotations

,ID,KEY
0,5061,D# minor
1,74680,A minor
2,299807,D# major
3,608572,D major
4,1492875,C minor
5,4857871,A# major
6,6529435,C minor


## Data Processing


### Part 1: Annotation Data Processing


## Part 2: Audio Data Processing
